In [ ]:
# Imports
%env TF_GPU_THREAD_MODE gpu_private
import tensorflow as tf # env TF_GPU_THREAD_MODE=gpu_private somehow makes training much faster
tf.config.experimental.set_memory_growth((tf.config.list_physical_devices('GPU'))[0], True)
import tensorflow.keras as keras, tensorflow.keras.layers as layers
import matplotlib.pyplot as plt
import os
import sys
import import_ipynb
import Dataset2d
import MomentMetrics
import custom_layers

In [ ]:
def evaluate(dataset, model, history, useYScale = False):
    """Evaluates a model
     param dataset: The dataset, whose test attribut will be used for testing
     param model: The model to be evaluated
     param history: history to be plotted
     param useYScale: whether to plot the yscale as symlog; Default: False 
    """
    metrics = model.evaluate(dataset.test, verbose=0)
    print("Evaluated Metrics for the Test-Set:")
    for name, value in zip(model.metrics_names, (metrics if hasattr(metrics, '__iter__') else [metrics])):
        print(f"Metric: {name}: {value}")
    plt.plot(history.history['loss'],'b') 
    plt.plot(history.history['val_loss'],'r') 
    if useYScale: plt.yscale('symlog')
    plt.show()
    
def build_model(num_out_dim = 4, input_shape = (8), num_out_vectors= 1, denseLayers = [], convLayers = []):
    """Builds the model. 
     param num_out_dim: number of dimensions defined by the labels, aka in 2D Sim available
     param input_shape: shape of model input
     param num_out_vectors: number of output vectors. Output shape will therefore be num_out_vectors x num_out_dim
     param denseLayers: list of number of neurons per dense layer 
     param convLayers: list of convLayers. Each convLayer consists of a list [num_filters, kernel_diameter, pool_diameter].
         The model is added a Conv2D(filters=num_filters, kernel_size=(kernel_diameter,kernel_diameter)).BatchNorm.MaxPool2D((pool_diameter,pool_diameter))
    """
    model = keras.Sequential(name="Testmodel_2D")
    model.add(layers.InputLayer(input_shape=input_shape))
    for i,c in enumerate(convLayers):
        nfilters, kernel_d, pool_d = c
        model.add(layers.Conv2D(filters=nfilters, kernel_size=(kernel_d,kernel_d), activation='relu',name=f"Conv_{i}"))
        model.add(layers.BatchNormalization(name=f"BN_{i}"))
        model.add(layers.MaxPool2D(pool_size=(pool_d, pool_d),name=f"Pool_{i}"))
    if(convLayers): model.add(layers.Flatten(name="Flatten_to_Dense")) # only flatten if convLayers is not empty
    for i,d in enumerate(denseLayers): 
        model.add(layers.Dense(d, activation='relu', name=f"Dense_{i}"))
    model.add(layers.Dense(num_out_vectors*num_out_dim, name="Final_Paths"))
    model.add(layers.Reshape((num_out_vectors,num_out_dim), name="Shape_to_paths"))
    model.add(custom_layers.PathNormalisation_layer(name="Normalise_paths"))
    model.add(custom_layers.Covariance_layer(name="Cov"))
    return model

def buildAndTrainModel(dataset, num_out_vectors, loss, epochs, metrics = [], printSummary=True, callbacks=[], denseLayers=[], convLayers = [], input_shape = (8)):
    """Builds and trains a model
     param dataset: Dataset used for training
     param num_out_vectors: number of dimensions defined by the labels, aka in 2D Sim available
     param loss: Loss to be applied during training
     param epochs: Target number of epochs used during training
     param metrics: List of metrics used in training. Default: []
     param printSummary: Whether to print the model summary. Default: True
     param callbacks: Callback functions forwarded to tf. Default: [], 
     param denseLayers: list of number of neurons per dense layer in the model. Default:[]
     param convLayers: list of convLayers. Each convLayer consists of a list [num_filters, kernel_diameter, pool_diameter]. Default: [] 
     param input_shape: shape of model input. Default: (8)
    """
    model = build_model(num_out_vectors = num_out_vectors,input_shape = input_shape, denseLayers = denseLayers, convLayers=convLayers) # define model
    if(printSummary): model.summary()
    model.compile(optimizer='adam', loss=loss, metrics=metrics)
    history = model.fit(dataset.train, epochs=epochs, batch_size=None, shuffle=False, callbacks=callbacks, 
                        validation_data=ds.test)
    evaluate(dataset, model, history)
    return model, history

In [ ]:
# Load, prepare and display the dataset
path="../Datasets/Dataset_square_boxes/"
ds = Dataset2d.Dataset2d(path, {"train_test_ratio":20, "step_size":10, "batch_size":64,\
        'render_images':True, 'render_poles':False})
print(ds)

In [ ]:
# Build and train the model
model, history = buildAndTrainModel(ds, epochs=2, 
                                    input_shape=(600,600,3), num_out_vectors=12,
                                    convLayers=[[3,3,6],[4,3,6],[5,3,4]],
                                    denseLayers=[10,10,20],
                                    callbacks = [
                                        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2) 
                                        #,tf.keras.callbacks.TensorBoard()
                                        ],
                                    printSummary=True, 
                                    loss = MomentMetrics.momentLoss_onlyExponent_onSigma, metrics=[])

In [ ]:
# Print examples of network output
## Green Arrows: label user input
## Orange Arrows: First proposed DoF
## Yellow Arrows: Second proposed DoF
def custom_func(feat, label, model, loss):
    return
    
Dataset2d.plotDatapointBatch(ds.test, numPlots=20, figsize=(5,5), model=model,useEigenvectors=False, useSigma=True, 
    loss=MomentMetrics.momentLoss_onlyExponent_onSigma, custom_func=custom_func, printTexts=False)

In [ ]:
# Store calculated model in default location
storagePath = "./model/"
if not os.path.exists(storagePath): os.makedirs(storagePath)
Dataset2d.storeModel(model, storagePath)
with open(storagePath+"about.txt", 'w') as file: # About File:
    datasetName = path.split("/")[-2] if path[-1] == '/' else path.split("/")[-1]
    about = f"Dataset: {datasetName}\nNumber of epochs: {history.params['epochs']}\n"+\
            f"\n\n{ds}\n\n"
    file.write(about)
    model.summary(print_fn=lambda x: file.write(x + '\n'))
    about2 = f"Metrics per epoch in training: {history.history}\n\n"
    file.write(about2)

In [ ]:
# Copy model from default location to namedModels
val = input("Name to store Model under (Warning, must contain _poles to work with poles): ")
if(val):
    import shutil, pathlib
    the_path = pathlib.Path(f"../JS_Simulation/namedModels/{val}")
    the_path.mkdir(exist_ok=True)
    for file in ["about.txt", "model.json", "model_weights.h5"]:
        shutil.copy(f"{storagePath}/{file}", f"{the_path}/{file}")